In [1]:
from Query import Query
import os
from glob import glob
import pandas as pd
from Tempo import DataHora

In [2]:
sql=Query('Netfeira','sqlserver','MOINHO','192.168.0.252')

In [3]:
data=DataHora()

data_atual=data.HoraAtual()

In [4]:
pd.set_option('display.max_columns',None)

pd.set_option('float_format','{:2f}'.format)

In [5]:
tabelas_df=sql.CriarTabela()

In [6]:
tabelas_df['Pagar'].columns

Index(['Data de Emissão', 'Data de Vencimento', 'Data de Pagamento', 'Título',
       'NFe', 'ID Fornecedor', 'Razão Social', 'Nome Fantasia', 'CNPJ',
       'ID Conta', 'Contas', 'ID Tipo Conta', 'Tipo de Conta',
       'ID Grupo Conta', 'Grupo', 'ID Situação', 'Situação do Título',
       'Título R$', 'Saldo R$', 'Valor Pago R$', 'Desconto R$', 'Total R$'],
      dtype='object')

In [7]:
len(tabelas_df['Pagar']['Título'].loc[(tabelas_df['Pagar']['Data de Vencimento'].dt.year==data_atual.year)&(tabelas_df['Pagar']['Data de Vencimento'].dt.month==data_atual.month)&(tabelas_df['Pagar']['ID Situação'].isin(['AB','PL']))].unique().tolist())

tabelas_df['Pagar']['Título R$'].loc[(tabelas_df['Pagar']['Data de Vencimento'].dt.year==data_atual.year)&(tabelas_df['Pagar']['Data de Vencimento'].dt.month==data_atual.month)&(tabelas_df['Pagar']['ID Situação'].isin(['AB','PL']))].sum()

1324689.5899999999

In [8]:
faturado=round(tabelas_df['Estatico']['Total Geral'].loc[(tabelas_df['Estatico']['Tipo de Operação']=='VENDAS')&(tabelas_df['Estatico']['Status do Pedido']!='EM ABERTO')&(tabelas_df['Estatico']['Data de Faturamento'].dt.year==data_atual.year)&(tabelas_df['Estatico']['Data de Faturamento'].dt.month==data_atual.month)].sum(),2)

faturado

864773.81

In [9]:
aberto=round(tabelas_df['Aberto']['Total Geral'].loc[(tabelas_df['Aberto']['Data do Pedido'].dt.year==data_atual.year)&(tabelas_df['Aberto']['Data do Pedido'].dt.month==data_atual.month)&(tabelas_df['Aberto']['Data do Pedido'].dt.day==data_atual.day)&(tabelas_df['Aberto']['Situação']=='AB')].sum())

aberto

71564

In [10]:
total=faturado+aberto

total

936337.81

In [11]:
base_contas=['CAIXA-CONSORCIO','DISTRIBUICAO DE LUCRO','FORNECEDORES','RECIFE','SALVADOR','BELEM','CAMBIO','CONSORCIO','LITORAL','NUMERARIO','PARANA']

base_contas

['CAIXA-CONSORCIO',
 'DISTRIBUICAO DE LUCRO',
 'FORNECEDORES',
 'RECIFE',
 'SALVADOR']

In [17]:
tabelas_df['Pagar']['Contas']=tabelas_df['Pagar']['Contas'].apply(lambda info: str(info).strip())

In [11]:
tabelas_df['Pagar']=tabelas_df['Pagar'].loc[~tabelas_df['Pagar']['Contas'].isin(base_contas)]

In [12]:
tabelas_df['Pagar'].columns

Index(['Data de Emissão', 'Data de Vencimento', 'Data de Pagamento', 'Título',
       'NFe', 'ID Fornecedor', 'Razão Social', 'Nome Fantasia', 'CNPJ',
       'ID Conta', 'Contas', 'Tipo de Conta', 'Grupo', 'Situação do Título',
       'Título R$', 'Saldo R$', 'Valor Pago R$', 'Desconto R$', 'Total R$'],
      dtype='object')

In [13]:
valor_pago=round(tabelas_df['Pagar']['Valor Pago R$'].loc[(tabelas_df['Pagar']['Data de Pagamento'].dt.year==data_atual.year)&(tabelas_df['Pagar']['Data de Pagamento'].dt.month==data_atual.month)].sum(),2)

valor_pago

65324.8

In [14]:
desconto=round(tabelas_df['Pagar']['Desconto R$'].loc[(tabelas_df['Pagar']['Data de Pagamento'].dt.year==data_atual.year)&(tabelas_df['Pagar']['Data de Pagamento'].dt.month==data_atual.month)].sum(),2)

desconto

0.0

In [15]:
despesa_rep=round(valor_pago/total,4)*100

despesa_rep

27.42

In [16]:
titulos_vencidos=len(tabelas_df['Pagar']['Título'].loc[(tabelas_df['Pagar']['Data de Vencimento'].dt.year==data_atual.year)&(tabelas_df['Pagar']['Data de Vencimento'].dt.month==data_atual.month)].unique().tolist())

titulos_pagos=len(tabelas_df['Pagar']['Título'].loc[(tabelas_df['Pagar']['Data de Pagamento'].dt.year==data_atual.year)&(tabelas_df['Pagar']['Data de Pagamento'].dt.month==data_atual.month)].unique().tolist())

titulos_emitidos=len(tabelas_df['Pagar']['Título'].loc[(tabelas_df['Pagar']['Data de Emissão'].dt.year==data_atual.year)&(tabelas_df['Pagar']['Data de Emissão'].dt.month==data_atual.month)].unique().tolist())

titulos_emitidos

33

In [17]:
tit_venc_df=tabelas_df['Pagar'][['Data de Vencimento','Título R$']].loc[(tabelas_df['Pagar']['Data de Vencimento'].dt.year==data_atual.year)&(tabelas_df['Pagar']['Data de Vencimento'].dt.month==data_atual.month)].groupby(['Data de Vencimento','Título R$'],as_index=False).sum()

valor_vencer=tit_venc_df['Título R$'].sum()

In [18]:
total_pago=valor_pago+desconto

total_pago

65324.8

In [19]:
tabelas_df['Receber'].columns

Index(['Data de Emissão', 'Data de Vencimento', 'Data de Pagamento', 'Título',
       'Serie', 'Tipo de Pagamento', 'ID Cliente', 'Razão Social',
       'Nome Fantasia', 'Matriz', 'Segmento', 'Situação', 'Valor',
       'Desconto R$', 'Multa R$', 'Juros R$', 'Abatimento R$', 'Taxa R$',
       'Pago R$'],
      dtype='object')

In [20]:
recebido=round(tabelas_df['Receber']['Pago R$'].loc[(tabelas_df['Receber']['Data de Pagamento'].dt.year==data_atual.year)&((tabelas_df['Receber']['Data de Pagamento'].dt.month==data_atual.month))].sum(),2)

recebido

122956.2

In [21]:
tabelas_df['Pagar'].columns

Index(['Data de Emissão', 'Data de Vencimento', 'Data de Pagamento', 'Título',
       'NFe', 'ID Fornecedor', 'Razão Social', 'Nome Fantasia', 'CNPJ',
       'ID Conta', 'Contas', 'Tipo de Conta', 'Grupo', 'Situação do Título',
       'Título R$', 'Saldo R$', 'Valor Pago R$', 'Desconto R$', 'Total R$'],
      dtype='object')

In [58]:
lista_valores=tabelas_df['Pagar'][['Valor Pago R$','Desconto R$','Total R$']].loc[(tabelas_df['Pagar']['Data de Pagamento'].dt.year==data_atual.year)&(tabelas_df['Pagar']['Data de Pagamento'].dt.month==data_atual.month)].sum().to_dict()

grupo_df=tabelas_df['Pagar'][['Grupo','Valor Pago R$','Desconto R$','Total R$']].loc[(tabelas_df['Pagar']['Data de Pagamento'].dt.year==data_atual.year)&(tabelas_df['Pagar']['Data de Pagamento'].dt.month==data_atual.month-1)].groupby(['Grupo'],as_index=False).sum()

grupo_df['Rep %']=grupo_df.apply(lambda info: round(info['Valor Pago R$']/lista_valores['Valor Pago R$'],4)*100,axis=1)

grupo_df

,Grupo,Valor Pago R$,Desconto R$,Total R$,Rep %
0,DESPESAS FIXAS,517542.310000,0.000000,517542.310000,792.260000
1,DESPESAS VARIAVEIS,519993.920000,37.340000,520031.260000,796.010000
2,INVESTIMENTOS,7046.520000,0.000000,7046.520000,10.790000


In [72]:
tp_conta=pd.DataFrame()

for g in grupo_df['Grupo'].tolist():
    
    vl_pago=grupo_df['Valor Pago R$'].loc[grupo_df['Grupo']==g].sum()
        
    temp_df=tabelas_df['Pagar'][['Tipo de Conta','Valor Pago R$','Desconto R$','Total R$']].loc[(tabelas_df['Pagar']['Data de Pagamento'].dt.year==data_atual.year)&(tabelas_df['Pagar']['Data de Pagamento'].dt.month==data_atual.month-1)&(tabelas_df['Pagar']['Grupo']==g)].groupby(['Tipo de Conta'],as_index=False).sum()
        
    temp_df.sort_values('Valor Pago R$',ascending=False,inplace=True,ignore_index=True)
    
    temp_df['Rep %']=round(temp_df['Valor Pago R$']/vl_pago,4)*100
    
    temp_df=temp_df.loc[temp_df['Rep %']>=5]
    
    tp_conta=pd.concat([tp_conta,temp_df],axis=0,ignore_index=True)
    
    pass

tp_conta.sort_values('Valor Pago R$',ascending=False,inplace=True,ignore_index=True)

tp_conta

,Tipo de Conta,Valor Pago R$,Desconto R$,Total R$,Rep %
0,DESPESAS COM VENDAS,326946.860000,37.340000,326984.200000,62.880000
1,DESPESAS COM PESSOAL,322387.650000,0.000000,322387.650000,62.290000
2,IMPOSTOS,185933.660000,0.000000,185933.660000,35.760000
3,DESPESAS ADMINISTRATIVAS,163798.370000,0.000000,163798.370000,31.650000
4,IMOBILIZADO,7046.520000,0.000000,7046.520000,100.000000


In [74]:
contas_df=pd.DataFrame()

for g in tp_conta['Tipo de Conta'].tolist():
    
    vl_pago=tp_conta['Valor Pago R$'].loc[tp_conta['Tipo de Conta']==g].sum()
        
    temp_df=tabelas_df['Pagar'][['Contas','Valor Pago R$','Desconto R$','Total R$']].loc[(tabelas_df['Pagar']['Data de Pagamento'].dt.year==data_atual.year)&(tabelas_df['Pagar']['Data de Pagamento'].dt.month==data_atual.month-1)&(tabelas_df['Pagar']['Tipo de Conta']==g)].groupby(['Contas'],as_index=False).sum()
        
    temp_df.sort_values('Valor Pago R$',ascending=False,inplace=True,ignore_index=True)
    
    temp_df['Rep %']=round(temp_df['Valor Pago R$']/vl_pago,4)*100
    
    temp_df=temp_df.loc[temp_df['Rep %']>=5]
    
    contas_df=pd.concat([contas_df,temp_df],axis=0,ignore_index=True)
    
    pass

contas_df.sort_values('Valor Pago R$',ascending=False,inplace=True,ignore_index=True)

contas_df

,Contas,Valor Pago R$,Desconto R$,Total R$,Rep %
0,COMISSAO,168382.280000,0.000000,168382.280000,51.500000
1,FRETE,151710.900000,0.000000,151710.900000,46.400000
2,SALARIOS,149332.110000,0.000000,149332.110000,46.320000
3,ICMS,83069.200000,0.000000,83069.200000,44.680000
4,INSS FUNCIONARIOS,64923.130000,0.000000,64923.130000,20.140000
5,COFINS,58134.070000,0.000000,58134.070000,31.270000
6,LUZ,40899.250000,0.000000,40899.250000,24.970000
7,VALE ALIMENTACAO/REFEICAO,30551.540000,0.000000,30551.540000,9.480000
8,ALUGUEL,29758.700000,0.000000,29758.700000,18.170000
9,IRPJ,20546.570000,0.000000,20546.570000,11.050000


In [23]:
def ValorAnterior(mes):
            
    if(mes==1):
        
        mes=12
        
        ano=data_atual.year-1
        
        pass
    
    else:
        
        mes-=1
        
        ano=data_atual.year
        
        pass
    
    total=tabelas_df['Pagar']['Valor Pago R$'].loc[(tabelas_df['Pagar']['Data de Pagamento'].dt.year==ano)&(tabelas_df['Pagar']['Data de Pagamento'].dt.month==mes)].sum()
    
    return total
        
    
    pass

In [24]:
def TotalVendido(mes):

    faturado=round(tabelas_df['Estatico']['Total Geral'].loc[(tabelas_df['Estatico']['Tipo de Operação']=='VENDAS')&(tabelas_df['Estatico']['Status do Pedido']!='EM ABERTO')&(tabelas_df['Estatico']['Data de Faturamento'].dt.year==data_atual.year)&((tabelas_df['Estatico']['Data de Faturamento'].dt.month==mes))].sum(),2)
    
    return faturado
    
    pass

In [28]:
mensal_df=tabelas_df['Pagar'].loc[tabelas_df['Pagar']['Data de Pagamento'].dt.year==data_atual.year]

mensal_df=mensal_df[['Data de Pagamento','Valor Pago R$']].groupby(['Data de Pagamento'],as_index=False).sum()

mensal_df['ID Mês']=mensal_df['Data de Pagamento'].dt.month

mensal_df['Mês']=mensal_df.apply(lambda info: data.Mes(info['ID Mês']),axis=1)

mensal_df=mensal_df[['ID Mês','Mês','Valor Pago R$']].groupby(['ID Mês','Mês'],as_index=False).sum()

mensal_df['Pago Anterior R$']=mensal_df['ID Mês'].apply(ValorAnterior)

mensal_df['Cresc %']=mensal_df.apply(lambda info: round(((info['Valor Pago R$']/info['Pago Anterior R$'])-1),4)*100,axis=1)

mensal_df['Vendas R$']=mensal_df['ID Mês'].apply(TotalVendido)

mensal_df['Rep %']=mensal_df.apply(lambda info: round(((info['Valor Pago R$']/info['Vendas R$'])),4)*100,axis=1)

mensal_df.sort_values('ID Mês',ascending=True,inplace=True)

mensal_df

,ID Mês,Mês,Valor Pago R$,Pago Anterior R$,Cresc %,Vendas R$,Rep %
0,1,JANEIRO,857825.000000,1043556.660000,-17.800000,3214772.920000,26.680000
1,2,FEVEREIRO,846418.800000,857825.000000,-1.330000,3259230.880000,25.970000
2,3,MARÇO,1027288.830000,846418.800000,21.370000,4443762.790000,23.120000
3,4,ABRIL,1065778.200000,1027288.830000,3.750000,3841883.070000,27.740000
4,5,MAIO,1064962.750000,1065778.200000,-0.080000,4151137.640000,25.650000
5,6,JUNHO,1044582.750000,1064962.750000,-1.910000,3614827.780000,28.900000
6,7,JULHO,65324.800000,1044582.750000,-93.750000,121731.610000,53.660000
